In [ ]:
!pip install -U -q  "faiss-cpu" "tiktoken" "sentence-transformers" "anthropic" "python-dotenv" "pypdf" "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "PyPDF2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc.pdf"

--2024-04-08 23:02:42--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1493834 (1.4M) [application/pdf]
Saving to: ‘data/ppc.pdf’

data/ppc.pdf        100%[===================>]   1.42M  1.24MB/s    in 1.2s    

2024-04-08 23:02:45 (1.24 MB/s) - ‘data/ppc.pdf’ saved [1493834/1493834]



In [ ]:
from google.colab import userdata

def write_to_file(filename, text):
    try:
        with open(filename, 'a+') as file:
            file.write('\n'+text)
    except Exception as e:
        print(f"Error writing to file: {e}")

write_to_file('prompts.txt', 'teste2')


class Settings:
  ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
  HF_TOKEN          = userdata.get('HF_TOKEN')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore


def extract_text_from_pdf():
  loader = PyPDFLoader("data/ppc.pdf")
  pages = loader.load_and_split()
  return pages


def split_into_chunks(pages):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000, # the character length of the chunk
    chunk_overlap = 200, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

  return text_splitter.transform_documents(pages)

def init_embeddings_model(embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'):

 return HuggingFaceEmbeddings(model_name=embed_model_id), embed_model_id

def create_vector_store(core_embeddings_model, documents, emdbed_model_id):
  store = LocalFileStore("./cache/")

  embedder = CacheBackedEmbeddings.from_bytes_store(
      core_embeddings_model, store, namespace=embed_model_id
  )

  vector_store = FAISS.from_documents(documents, embedder)

  return vector_store

def save_vector_store(vector_store):
  vector_store.save_local('ppc-index')

In [ ]:
pages = extract_text_from_pdf()
documents = split_into_chunks(pages)

In [ ]:
core_embeddings_model, embed_model_id = init_embeddings_model('sentence-transformers/distiluse-base-multilingual-cased-v1')

vector_store = create_vector_store(core_embeddings_model, documents, embed_model_id)


.gitattributes:   0%|          | 0.00/744 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
# query = "Qual o limite de horas para validar atividades complementares de estágios nao obrigatórios?"
# embedding_vector = core_embeddings_model.embed_query(query)
# docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)

# for page in docs:
#   print(page.page_content)

# Testando com Anthropic Claude

In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
import anthropic
from groq import Groq


client = Groq(api_key = settings.GROQ_API_KEY)

def get_groq_response(prompt, model_id = "mixtral-8x7b-32768"):
  response = client.chat.completions.create(
      model="mixtral-8x7b-32768",
      messages=[
          {
              "role": "system",
              "content": "You are a helpful assistant."
          },
          {
              "role": "user",
              "content": prompt
          }
      ],
      max_tokens=4096
  )

  return(response.choices[0].message.content)

def build_prompt(query, context):
  return f"""Human:\n\n
  Voce é um assistente academico especializado em tirar dúvidas de alunos de ciencia da computação da UFFS.
  Responda, em portugues, com base no contexto:
  {context}
  Responda apenas se encontrar resposta no contexto.
  Pergunta: {query}
  A resposta deve seguir o padrão:
  - Pergunta
  R: Resposta (trecho de referencia para resposta)

  Assistant:
  """

def get_claude_response(prompt):
  client = anthropic.Anthropic(
      api_key = settings.ANTHROPIC_API_KEY
    )
  message = client.messages.create(
      model="claude-instant-1.2",
      max_tokens=200,
      temperature=0.1,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": prompt
                  }
              ]
          }
      ]
  )
  return message.content[0].text

def get_context(query):
  embedding_vector = core_embeddings_model.embed_query(query)
  docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)

  context = ''
  for page in docs:
    context += page.page_content

  return context

def get_answer(query, model_id = 'claude-instant-1.2'):

  context = get_context(query)
  prompt = build_prompt(context, query)
  write_to_file('prompts.txt', prompt)
  if model_id.startswith('claude'):
    return get_claude_response(prompt, model_id)

  return get_groq_response(prompt, model_id)




In [ ]:
queries = [
    "Qual a ementa de algebra linear?",
    "Qual o limite de horas de atividades curriculares de Pesquisa?",
    "Quantas horas de ACC devo ter para me formar?",
    "Preciso das referencias para a disciplina Engenharia de Software II",
    "Qual o limite de horas para validar atividades complementares de estágios nao obrigatórios?",
    "Qual a ementa de Grafos?"
]

# for query in queries:
#   print()
#   response = get_answer(query)
#   print(response)

## Testando com mixtral-8x7b-32768

In [ ]:
from groq import Groq

client = Groq(api_key = settings.GROQ_API_KEY)

def get_groq_response(prompt, model_id = "mixtral-8x7b-32768"):
  response = client.chat.completions.create(
      model="mixtral-8x7b-32768",
      messages=[
          {
              "role": "system",
              "content": "You are a helpful assistant."
          },
          {
              "role": "user",
              "content": prompt
          }
      ],
      max_tokens=4096
  )

  return(response.choices[0].message.content)




In [ ]:
for query in queries:
  context = get_context(query)

  prompt = build_prompt(query, context).split('Human:')[1].replace('Assistant:', '')

  completion = get_groq_response(prompt)
  print(query)
  print(completion)

Qual a ementa de algebra linear?
- Pergunta
R: Matrizes, Determinantes, Sistemas de equações lineares, Espaços vetoriais, Transformações lineares, Espaços com produto interno, Autovalores e autovetores, Diagonalização. (Referência: Projeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado)
Qual o limite de horas de atividades curriculares de Pesquisa?
Pergunta: Qual o limite de horas de atividades curriculares de Pesquisa?

R: As atividades curriculares de pesquisa estão incluídas no Grupo I de Atividades Curriculares Complementares (ACCs) e têm um limite total de 300 horas ao longo do curso. Isso equivale a 360 horas-aula. (Referência: Projeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado, p. 8.7)
Quantas horas de ACC devo ter para me formar?
- Pergunta
R: Para se formar, o estudante deve ter 300 horas de Atividades Curriculares Complementares (ACC) (trecho de referência: "As ACCs constituem ações que visam a complementação do processo e

## Testando com llama2-70b

In [ ]:
for query in queries:
  context = get_context(query)

  prompt = build_prompt(query, context).split('Human:')[1].replace('Assistant:', '')

  completion = get_groq_response(prompt, 'llama2-70b-4096')
  print(query)
  print(completion)

Qual a ementa de algebra linear?
- Pergunta
R: "Matrizes. Determinantes. Sistemas de equações lineares. Espaços vetoriais. Transformações lineares. Espaços com produto interno. Autovalores e autovetores. Diagonalização." (Referência: Projeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado)
Qual o limite de horas de atividades curriculares de Pesquisa?
- Pergunta
R: The limit of hours for curricular activities of Research is 300 hours (Projeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado 201).
Quantas horas de ACC devo ter para me formar?
- Pergunta
R: As Atividades Curriculares Complementares (ACCs) devem ser desenvolvidas com carga horária de 300 horas, ou equivalentemente, 360 horas-aula, e são consideradas obrigatórias para a integralização do currículo (Lei de Diretrizes e Bases da Educação Nacional 9.394/1996, Resolução CNE/CES 136/2012).
Preciso das referencias para a disciplina Engenharia de Software II
Pergunta: Preciso das ref

In [ ]:
!pip install transformers
!pip install einops accelerate bitsandbytes
!pip install sentence_transformers
!pip install git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-mar2wap0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-mar2wap0
  Resolved https://github.com/huggingface/peft.git to commit 26726bf1ddee6ca75ed4e1bfd292094526707a78
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.10.1.dev0-py3-none-any.whl size=200819 sha256=0509272882659c252dada49c167efaf3ef50b82b6bb42b486226abf5cd766e4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-z1r68uz1/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0
    Uninstalling peft-0.4.0:
      Successfully uninstalled peft-0.4.0


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# # import torch
# # import transformers
# # from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
# # from peft import PeftModel, PeftConfig

# # model_id = 'recogna-nlp/bode-7b-alpaca-pt-br'

# # config = PeftConfig.from_pretrained(model_id)


# # model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, return_dict=True, load_in_8bit=True, device_map='auto')

# # model.eval()



# import torch
# import transformers

# model_id = "meta-llama/Llama-2-13b-chat-hf"

# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model_config = transformers.AutoConfig.from_pretrained(
#     model_id
# )

# model = transformers.AutoModelForCausalLM.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     config=model_config,
#     quantization_config=bnb_config,
#     device_map='auto'
# )

# model.eval()

In [ ]:
# tokenizer = transformers.AutoTokenizer.from_pretrained(
#     model_id
# )

In [ ]:
# generate_text = transformers.pipeline(
#     model=model,
#     tokenizer=tokenizer,
#     task="text-generation",
#     return_full_text=True,
#     temperature=0.1,
#     max_new_tokens=512
# )

In [ ]:
# from langchain.llms import HuggingFacePipeline

# llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
from transformers import GenerationConfig


def generate_prompt(instruction, input=None):
    if input:
        return f"""Abaixo está uma instrução que descreve uma tarefa, juntamente com contexto. Escreva uma resposta que complete adequadamente o pedido, com base no contexto.

### Instrução:
Responda a pergunta com base no contexto provido: {instruction}

### Contexto:
{input}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Resposta:"""

generation_config = GenerationConfig(
    temperature=0.0,
    num_beams=2,
    do_sample=True
)

def evaluate(prompt):
    # prompt = generate_prompt(instruction, input)
    # inputs = tokenizer(prompt, return_tensors="pt")
    # input_ids = inputs["input_ids"].cuda()
    generation_output = llm(prompt)
    return generation_output.split("### Resposta:")[1].strip()
    # for s in generation_output.sequences:
    #     output = tokenizer.decode(s)
    #     print("Resposta:", output.split("### Resposta:")[1].strip())

def get_answer(query):
  embedding_vector = core_embeddings_model.embed_query(query)
  docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)

  context = ''
  for page in docs:
    context += page.page_content



  prompt = generate_prompt(query, context)
  return prompt
  # write_to_file('prompts-llama.txt', prompt)
  response = evaluate(prompt)

  return response




In [ ]:
queries = [
    "Qual a ementa de algebra linear?",
    "Qual o limite de horas de atividades curriculares de Pesquisa?",
    "Quantas horas de ACC devo ter para me formar?",
    "Preciso das referencias para a disciplina Engenharia de Software II",
    "Qual o limite de horas para validar atividades complementares de estágios nao obrigatórios?",
    "Qual a ementa de Grafos?"
]
queries = [queries[0]]
for query in queries:
  completion = get_answer(query)
  print(query)
  print(completion)

Qual a ementa de algebra linear?
Abaixo está uma instrução que descreve uma tarefa, juntamente com contexto. Escreva uma resposta que complete adequadamente o pedido, com base no contexto.

### Instrução:
Responda a pergunta com base no contexto provido: Qual a ementa de algebra linear?

### Contexto:
ed. New York: John Wiley & Sons, 1967. v. 1.
______. Calculus: one-variable calculus, with an introduction to linear algebra. 2. ed. New
York: John Wiley & Sons, 1967. v. 2.
LARSON, R.; HOSTETLER, R. P.; EDWARDS, B. H. Cálculo. 8. ed. São Paulo: McGraw
Hill, 2006. 1 v.
______. Cálculo. 8 ed. São Paulo: McGraw Hill, 2006. 2 v.
SALAS, H. E. Cálculo. 9. ed. Rio de Janeiro: LTC, 2005. 1 v.
______. Cálculo. 9. ed. Rio de Janeiro: LTC, 2005. 2 v.
SIMMONS, G. F. Cálculo com geometria analítica. São Paulo: McGraw Hill, 1987. 1 v.
______. Cálculo com geometria analítica. São Paulo: McGraw Hill, 1987. 2 v.
Projeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado 68MINISTÉRIO 

In [ ]:
# retriever = vector_store.as_retriever()

In [ ]:
# from langchain.chains import RetrievalQA
# from langchain.callbacks import StdOutCallbackHandler

# handler = StdOutCallbackHandler()

# qa_with_sources_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     callbacks=[handler],
#     return_source_documents=True
# )

In [ ]:
# qa_with_sources_chain({"query" : "Qual a ementa de geometria analíica?"})